# (공부) 네임스페이스 & 연산자오버로딩 & 도움말 추가하기
> 작성완료

- toc:true
- branch: master
- badges: true
- comments: true
- author: kimha02
- categories: [Python]

----

### 예제1

In [1]:
class Testclass1: 
    x=0

In [2]:
Testclass1.x

0

In [3]:
a=Testclass1()

In [4]:
a.x

0

`–` Testclass1.x를 수정하면 a.x가 강제로 수정된다. 

In [5]:
Testclass1.x=100

In [6]:
a.x

100

`-` a.x를 수정한다고 하여 Testclass1.x가 강제로 수정되는 것은 아님

In [7]:
a.x=200

In [8]:
Testclass1.x

100

In [9]:
a.x

200

`-` 이건 왜이러지?

In [10]:
Testclass1.x=300

In [11]:
a.x

200

`-` 아래의 상황과 비슷하다. 

In [12]:
## [code1]
x=39
def nextyear():
    y=x+1
    print(x,y)
nextyear()

39 40


In [13]:
## [code2]
x=39
def nextyear():
    y=x+1
    print(x,y)
    x=0
nextyear()

UnboundLocalError: local variable 'x' referenced before assignment

`–` [code1]은 잘 실행되는 코드다.

`-` [code2]는 실행되지 않는 코드다.

`-` [code2]와 [code1]의 차이점은 x=0이라는 코드가 추가로 포함되었는지 유무다. 

`–` (헛소리) x=0 이 잘못된 코드다!! 이걸 실행하는 과정에서 문제가 생겼다!! 

`-` (올바른소리) code1에서는 x는 global variable, code2에서는 x가 local variable 이라서 생기는 문제점이다. 

In [14]:
## [code2]
x=39
def nextyear():
    x=0
    y=x+1
    print(x,y)
nextyear()

0 1


In [15]:
x

39

`–` 다시 우리의 예제로 돌아오자.
```python
### 시점1
class Testclass1: 
    x=0
### 시점2
a=Testclass1()
### 시점3
Testclass1.x=100
### 시점4
a.x=200
### 시점5
Testclass1.x=300
```

| |시점1 |시점2 |시점3 | 시점4 | 시점5|
|:-:|:-:|:-:|:-:|:-:|:-:|
|Testclass1.x|0| 0| 100| 100| 300|
|a.x| 값없음     | 0| 100| 200| 200|
|a.x의 속성| -     | class| class| instance | instance|

`–` a.x가 클래스로부터 물려받은 속성인지 (그래서 클래스와 연결되어있는지) 아니면 instance가 독자적으로 가지고 있는 속성인지 어떻게 알 수 있을까? 

In [16]:
### 시점1
class Testclass1: 
    x=0
print('시점1',Testclass1.x)
### 시점2
a=Testclass1()
print('시점2',Testclass1.x,a.x,a.__dict__)
### 시점3
Testclass1.x=100
print('시점3',Testclass1.x,a.x,a.__dict__)
### 시점4
a.x=200
print('시점4',Testclass1.x,a.x,a.__dict__)
### 시점5
Testclass1.x=300
print('시점5',Testclass1.x,a.x,a.__dict__)

시점1 0
시점2 0 0 {}
시점3 100 100 {}
시점4 100 200 {'x': 200}
시점5 300 200 {'x': 200}


------

### 예제2

In [18]:
x=11 ## 전역변수 ... A

def f():
    x=22 ## 함수 f안에 설정된 지역변수 
    print(x) ## 전역에 x=11 있지만 함수안에 x=22가 있으므로 x=22를 사용. --> 22출력됨 

def g():
    print(x) ## 함수 g안에 x를 찾아봤는데 없음 --> 전역에서 x를 찾음 --> x=11 --> 11출력함. 

class Testclass2: 
    x=33 ## 클래스 변수 ... B
    def m1(self):
        x=44 ## 메소드 변수 ... C
    def m2(self):
        self.x=44 ## 인스턴스 변수 ... D 

`-` 결과를 관찰하고 해석해보자. 

In [19]:
print(x)

11


> Note: 전역변수 출력

In [20]:
f()

22


> Note: f에서 설정된 지역변수 22가 출력됨

In [22]:
x

11

> Note: f내의 지역변수를 사용하여도 전역변수는 변하지 않음. (함수내부에서 선언된 x=22는 함수외부에 영향을 주지못함)

In [23]:
g()

11


> Note: g에서 설정된 지역변수가 따로 없으므로 전역변수 출력

In [24]:
x,Testclass2.x

(11, 33)

> Note: 전역변수 x와 클래스오브젝트에 설정된 변수 x 

In [25]:
a=Testclass2()
(x,Testclass2.x,a.x),a.__dict__

((11, 33, 33), {})

> Note: 전역변수, 클래스 오브젝트내의 변수, 인스턴스내의 변수. `a.__dict__`의 결과로 보아 인스턴스내의 변수는 클래스 오브젝트내의 변수를 빌려쓰고 있다. 

In [26]:
Testclass2.x=200
(x,Testclass2.x,a.x),a.__dict__

((11, 200, 200), {})

> Note: 클래스오브젝트에서 변수를 고치면 인스턴스에 영향을 미침

In [27]:
a.m1()
(x,Testclass2.x,a.x),a.__dict__

((11, 200, 200), {})

> Note: 메소드 `m1`내에서 선언된 x=44라는 선언은 아무것도 변화시킬수 없음. 

In [28]:
a.m2()
(x,Testclass2.x,a.x),a.__dict__

((11, 200, 44), {'x': 44})

> Note: 메소드 `m2`에 있는 self.x는 결국 a.x라는 의미이고, 이 선언은 클래스오브젝트 내의 변수와 독립적으로 인스턴스오브젝트 내에서 통용되는 변수를 선언하는 것임. 이 선언의 결과는 `a.__dict__`의 출력결과에서도 확인가능.

In [29]:
Testclass2.x=300
(x,Testclass2.x,a.x),a.__dict__

((11, 300, 44), {'x': 44})

> Note: 이제는 a.x와 Testclass2.x 는 분리된 상태이므로, Testclass2.x의 값을 바꾸어도 a.x에는 값의 변화가 없음. 

`-` 전역변수 > 클래스변수 > 인스턴스변수 > 메소드변수  
내용을 모르고 사용한다면 예상하지 못한 오류가 발생할 수 있으므로 조심해서 사용하자. 

----

## 연산자 오버로딩 

`-` 아래의 코드를 관찰하자. 

In [30]:
1+1

2

`-` 생각해보니까 `1`은 int class 에서 생성된 인스턴스이다. 

`-` 코드를 관찰하니 instance와 instance를 `+`라는 연산이 연결하는 형태임. 

In [34]:
class Student: 
    def __init__(self,age=20.0,semester=1):
        self.age=age
        self.semester=semester
    def __add__(self,val):
        # val==0: 휴학 
        # val==1: 등록
        if val==0: 
            self.age=self.age +0.5 
        elif val==1: 
            self.age=self.age+0.5
            self.semester=self.semester+1
        return self   ### return을 통해 guebin+1도 Student Type이 된다
    def __repr__(self):
        return '나이: %s \n학기: %s' % (self.age,self.semester)

In [35]:
guebin=Student()

In [36]:
guebin.age

20.0

In [37]:
guebin.semester

1

In [38]:
guebin

나이: 20.0 
학기: 1

In [39]:
type(guebin)

__main__.Student

In [40]:
guebin+1

나이: 20.5 
학기: 2

In [41]:
type(guebin+1)

__main__.Student

In [42]:
guebin+0

나이: 21.5 
학기: 3

In [43]:
guebin+0+0+0+0+1+0+1

나이: 25.0 
학기: 5

`-` 연산자 오버로드 핵심아이디어 
- 클래스가 일반 파이썬 연산을 재정의하는 것  
- 여기에서 연산은 단순히 더하기 빼기를 의미하는게 아니라, `print()`, `+`, `[0]` 와 같은 파이썬 내장문법을 모두 포괄하는 개념이라 이해하는 것이 옳다. 

In [44]:
guebin[0]

TypeError: 'Student' object is not subscriptable

In [45]:
class Student2(Student):
    def __getitem__(self,index):
        return [self.age,self.semester][index]

In [46]:
hynn=Student2()

In [47]:
hynn+1+1+0+0

나이: 22.0 
학기: 3

In [48]:
hynn[0]

22.0

In [49]:
hynn[1]

3

In [50]:
hynn[:]

[22.0, 3]

`-` 연산자 오버로딩을 이해하면 파이썬 전반에 대한 이해폭이 넓어진다. 

In [51]:
import pandas as pd 

In [52]:
df=pd.DataFrame({'age':[20,21.5],'semester':[1,2]})

In [53]:
df.iloc[:,0]

0    20.0
1    21.5
Name: age, dtype: float64

-------

## 도움말 작성방법 

`-` 넘파이의 경우 아래와 같이 도움말이 잘 작성되어 있다. 

In [55]:
import numpy as np
a=np.array([1,2,3])
a?

Type:            ndarray
String form:     [1 2 3]
Length:          3
File:            ~/anaconda3/envs/py38r40/lib/python3.8/site-packages/numpy/__init__.py
Docstring:       <no docstring>
Class docstring:
ndarray(shape, dtype=float, buffer=None, offset=0,
        strides=None, order=None)

An array object represents a multidimensional, homogeneous array
of fixed-size items.  An associated data-type object describes the
format of each element in the array (its byte-order, how many bytes it
occupies in memory, whether it is an integer, a floating point number,
or something else, etc.)

Arrays should be constructed using `array`, `zeros` or `empty` (refer
to the See Also section below).  The parameters given here refer to
a low-level method (`ndarray(...)`) for instantiating an array.

For more information, refer to the `numpy` module and examine the
methods and attributes of an array.

Parameters
----------
(for the __new__ method; see Notes below)

shape : tuple of ints
    Shape of cr

`-` 하지만 우리는? 

In [56]:
hynn?

Type:        Student2
String form:
나이: 22.0 
학기: 3
Docstring:   <no docstring>


`-` 우리도 도움말을 작성하고 싶다. 

In [57]:
class Student2(Student):
    '''
    Student2는 Student의 개선 
    
    # Student 클래스의 기능 
    1. 출력기능 (__repr__)
    2. 연산기능 (__add__): 학기와 나이를 카운트 
    Examples
    --------
    >>> hynn=Student2()
    >>> hynn+1
    나이: 20.5 
    학기: 2
    
    # Student2에서 추가된 기능 
    1. 인덱싱
    '''
    def __getitem__(self,index):
        return [self.age,self.semester][index]

In [58]:
hynn=Student2()

In [59]:
hynn?

Type:        Student2
String form:
나이: 20.0 
학기: 1
Docstring:  
Student2는 Student의 개선 

# Student 클래스의 기능 
1. 출력기능 (__repr__)
2. 연산기능 (__add__): 학기와 나이를 카운트 
Examples
--------
>>> hynn=Student2()
>>> hynn+1
나이: 20.5 
학기: 2

# Student2에서 추가된 기능 
1. 인덱싱


In [60]:
hynn=Student2(21,1)

In [61]:
hynn

나이: 21 
학기: 1

In [62]:
hynn?

Type:        Student2
String form:
나이: 21 
학기: 1
Docstring:  
Student2는 Student의 개선 

# Student 클래스의 기능 
1. 출력기능 (__repr__)
2. 연산기능 (__add__): 학기와 나이를 카운트 
Examples
--------
>>> hynn=Student2()
>>> hynn+1
나이: 20.5 
학기: 2

# Student2에서 추가된 기능 
1. 인덱싱


---------

## self에 대한 진실

`–` 사실 이름이 self가 아니어도 된다. 

In [ ]:
class MooYaHo: 
    def __init__(a):
        a.text='mooyaho'

In [ ]:
moo1=MooYaHo()

In [ ]:
moo1.text

`–` 일반적으로 사람들이 self를 많이 쓴다.  
a는 간단하게 정의할 때 많이 쓰이기 때문에 향후에 헛갈릴 수도 있다. 